# DATA643 
Harpreet Kaur Shoker

## Briefly describe the recommender system that you’re going to build out from a business perspective
In this project, i have build a recommender system for movies. I have created a sample dataset with 10 users and 6 movie. Have created a sparse dataset so that model will provide better estimates.

In [225]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

# Load your data into R

In [226]:
df = pd.read_csv("https://raw.githubusercontent.com/Harpreet1984/DATA643/master/Proj1/data/data.csv")
df.head(5)

,User,Movie1,Movie2,Movie3,Movie4,Movie5,Movie6
0,User1,NaN,NaN,NaN,4.0,NaN,4.0
1,User2,4.0,5.0,4.0,3.0,2.0,3.0
2,User3,NaN,5.0,NaN,NaN,NaN,5.0
3,User4,5.0,4.0,NaN,NaN,NaN,5.0
4,User5,4.0,NaN,2.0,NaN,2.0,5.0


In [227]:
df_long = pd.melt(df, id_vars='User').dropna()
raw_avg = df_long["value"].mean()
raw_avg

movie_mean = df.mean(axis=0).dropna()
movie_mean

user_mean = df.mean(axis=1).dropna()
user_mean

0    4.000000
1    3.500000
2    5.000000
3    4.666667
4    3.250000
5    3.500000
6    3.333333
7    4.000000
8    3.500000
9    3.666667
dtype: float64

## Break your ratings into separate training and test datasets

In [228]:
df_long.head(20)
df_long_train, df_long_test = train_test_split(df_long, test_size =0.3)

df_long_train.head(10)
df_long_test.head(10)

,User,variable,value
1,User2,Movie1,4.0
16,User7,Movie2,4.0
4,User5,Movie1,4.0
44,User5,Movie5,2.0
28,User9,Movie3,1.0
8,User9,Movie1,4.0
59,User10,Movie6,3.0
53,User4,Movie6,5.0
25,User6,Movie3,3.0
30,User1,Movie4,4.0


## Using the Training data, Calculate the raw average(mean) rating for every user-item combination.

In [229]:
df_short_train = df_long_train.pivot_table(index=['User'], columns='variable')

df_train_movie_mean = df_short_train.mean(axis=0).dropna()
print (str(df_train_movie_mean))

df_train_user_mean = df_short_train.mean(axis=1).dropna()
print (str(df_train_user_mean))

       variable
value  Movie1      4.250000
       Movie2      4.200000
       Movie3      3.666667
       Movie4      3.333333
       Movie5      2.500000
       Movie6      4.285714
dtype: float64
User
User1     4.000000
User10    3.800000
User2     3.333333
User3     5.000000
User4     4.500000
User5     3.500000
User6     3.666667
User7     3.200000
User8     4.000000
User9     4.500000
dtype: float64


## Calculate the RMSE for raw average for both your training data and test data

In [230]:
raw_avg_train = [df_long_train.mean().dropna()]
print ("Raw average for the training set" + str(raw_avg_train[0]))

df_train_movie_mean = df_train_movie_mean - raw_avg_train[0][0]
df_train_user_mean = df_train_user_mean - raw_avg_train[0][0]

df_long_train_pred =  len(df_long_train.index) * raw_avg_train
rmse_1_training = sqrt(mean_squared_error(df_long_train["value"], df_long_train_pred))
df_long_test_pred = len (df_long_test.index ) * raw_avg_train
rmse_1_test = sqrt(mean_squared_error(df_long_test["value"], df_long_test_pred))
print ("Training RMSE using Raw Avg :- " + str(rmse_1_training) + " Test RMSE:- " + str(rmse_1_test))

Raw average for the training setvalue    3.807692
dtype: float64
Training RMSE using Raw Avg :- 1.0748606624986248 Test RMSE:- 1.1087334850307364



## From the raw averages, and the appropriate user and item biases, calculate the baseline predictors for every user-item combination


In [231]:
bias_dict = {}
for userInd in df_train_user_mean.index :
    for movieInd in df_train_movie_mean.index :
        mapIndex = str(userInd) + str(movieInd[1])
        mapValue = raw_avg_train[0][0] + df_train_user_mean[userInd] + df_train_movie_mean[movieInd]
        if mapValue > 5 :
            mapValue = 5
        elif mapValue < 1:
            mapValue = 1
        bias_dict[mapIndex] = mapValue

bias_dict

{'User10Movie1': 4.2423076923076923,
 'User10Movie2': 4.1923076923076925,
 'User10Movie3': 3.6589743589743589,
 'User10Movie4': 3.3256410256410258,
 'User10Movie5': 2.4923076923076923,
 'User10Movie6': 4.2780219780219779,
 'User1Movie1': 4.4423076923076925,
 'User1Movie2': 4.3923076923076927,
 'User1Movie3': 3.858974358974359,
 'User1Movie4': 3.525641025641026,
 'User1Movie5': 2.6923076923076925,
 'User1Movie6': 4.4780219780219781,
 'User2Movie1': 3.775641025641026,
 'User2Movie2': 3.7256410256410262,
 'User2Movie3': 3.1923076923076925,
 'User2Movie4': 2.8589743589743595,
 'User2Movie5': 2.025641025641026,
 'User2Movie6': 3.8113553113553116,
 'User3Movie1': 5,
 'User3Movie2': 5,
 'User3Movie3': 4.8589743589743595,
 'User3Movie4': 4.5256410256410255,
 'User3Movie5': 3.6923076923076925,
 'User3Movie6': 5,
 'User4Movie1': 4.9423076923076925,
 'User4Movie2': 4.8923076923076927,
 'User4Movie3': 4.3589743589743595,
 'User4Movie4': 4.0256410256410255,
 'User4Movie5': 3.1923076923076925,
 'Use

## Calculate RMSE for the baseline predictors for Training

In [232]:
train_pred_value = []
train_actual_value = []
for index, row in df_long_train.iterrows() :
    train_actual_value.append(row["value"])
    
    mapKey = str(row["User"] + str(row["variable"]))
    if mapKey in bias_dict.keys():
        train_pred_value.append(bias_dict[mapKey])
    else :
        train_pred_value.append(row["value"])
        

#train_pred_value   
#train_actual_value
rmse_2_training = sqrt(mean_squared_error(train_actual_value, train_pred_value))
print ("Training RMSE using Model:- " + str(rmse_2_training))

Training RMSE using Model:- 0.8308722727573618


## Calculate RMSE for the baseline predictors for Test

In [233]:
test_pred_value = []
test_actual_value = []
for index, row in df_long_test.iterrows() :
    test_actual_value.append(row["value"])
    
    mapKey = str(row["User"] + str(row["variable"]))
    if mapKey in bias_dict.keys():
        test_pred_value.append(bias_dict[mapKey])
    else :
        test_pred_value.append(row["value"])
rmse_2_test = sqrt(mean_squared_error(test_actual_value, test_pred_value))
print ("Test RMSE using Model:- " + str(rmse_2_test))

Test RMSE using Model:- 1.1267544238033511
